In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16,ResNet50,MobileNetV2,InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [2]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\Processed Data\Train',
                                target_size=(80,80),batch_size=8,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\Processed Data\Train',
                                target_size=(80,80),batch_size=8,class_mode='categorical', subset='validation')

Found 54336 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\Processed Data\Test',
                                target_size=(80,80),batch_size=8,class_mode='categorical')

Found 16979 images belonging to 2 classes.


In [4]:
mobilenet = MobileNetV2(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
mobilenet_add = mobilenet.output
mobilenet_add = Flatten()(mobilenet_add)
mobilenet_add = Dense(64, activation='relu')(mobilenet_add)
mobilenet_add = Dropout(0.5)(mobilenet_add)
mobilenet_add = Dense(2,activation= 'softmax')(mobilenet_add)

mobilenet_model = Model(inputs=mobilenet.input, outputs= mobilenet_add)
for layer in mobilenet.layers:
    layer.trainable = False

9406464/9406464 [==============================] - 2s 0us/step


In [5]:
mobilenet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 40, 40, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 40, 40, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 40, 40, 32)   0           ['bn_Conv1[0][0]']               
                                                                                              

In [6]:
checkpoint = ModelCheckpoint(r'D:\Processed Data\Drowsiness Detection Models\mobilenetmodel.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [7]:
mobilenet_model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

mobilenet_model.fit_generator(train_data,steps_per_epoch=train_data.samples//8,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//8,
                   callbacks=callbacks,
                    epochs=20)

C:\Users\hp\AppData\Local\Temp\ipykernel_26300\3508801178.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mobilenet_model.fit_generator(train_data,steps_per_epoch=train_data.samples//8,


Epoch 1/20
6792/6792 [==============================] - ETA: 0s - loss: 0.3437 - accuracy: 0.8456
Epoch 1: val_loss improved from inf to 0.34013, saving model to D:\Processed Data\Drowsiness Detection Models\mobilenetmodel.h5
6792/6792 [==============================] - 3255s 473ms/step - loss: 0.3437 - accuracy: 0.8456 - val_loss: 0.3401 - val_accuracy: 0.8595 - lr: 0.0010
Epoch 2/20
6792/6792 [==============================] - ETA: 0s - loss: 0.2649 - accuracy: 0.8903
Epoch 2: val_loss improved from 0.34013 to 0.32274, saving model to D:\Processed Data\Drowsiness Detection Models\mobilenetmodel.h5
6792/6792 [==============================] - 4021s 592ms/step - loss: 0.2649 - accuracy: 0.8903 - val_loss: 0.3227 - val_accuracy: 0.8702 - lr: 0.0010
Epoch 3/20
6792/6792 [==============================] - ETA: 0s - loss: 0.2418 - accuracy: 0.9016
Epoch 3: val_loss improved from 0.32274 to 0.31555, saving model to D:\Processed Data\Drowsiness Detection Models\mobilenetmodel.h5
6792/6792 [=

KeyboardInterrupt: 